# Import Statements #

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import ipywidgets as widgets
import trimesh as tm
import scipy as sc
import numpy as np
import pandas as pd
from itertools import cycle
import skcriteria as sk
from skcriteria.agg import similarity  # here lives TOPSIS
from skcriteria.pipeline import mkpipe  # this function is for create pipelines
from skcriteria.preprocessing import invert_objectives, scalers
pv.set_jupyter_backend('trame')

#Display full array without truncation
np.set_printoptions(threshold=np.inf)

## 01. Loading the base object ##

We are loading the base .obj file here to create voxels. The voxel size is considered here is 3x3x3m and the base object is extruded to 2m so that a grid of only 1 voxel in depth is created. In the Hub implementation we can consider adding code to create a 2d voxel grid with the current 3d implementation

In [2]:
vs = 3
unit = [vs, vs, vs]
tol = 1e-09
mesh_path = os.path.relpath(r"C:\Users\aditya.soman\source\repos\Loci_Python\Obj_Data\floorobject.obj")
mesh = tg.geometry.load_mesh(mesh_path)


In [3]:
sample_cloud, ray_origins = tg.geometry.mesh_sampling(mesh, unit, multi_core_process=False, return_ray_origin = True, tol=tol)
lattice = sample_cloud.voxelate(unit, closed=True)

In [5]:
# initiating the plotter
p = pv.Plotter(notebook=True) # ITK plotter for interactivity within the python notebook (itkwidgets library is required)


# fast visualization of the point cloud
#sample_cloud.fast_notebook_vis(p)

# fast visualization of the lattice
lattice.fast_notebook_vis(p)

# adding the base mesh: light blue
mesh = pv.read(mesh_path)
#p.add_mesh(mesh, color='#abd8ff')

# adding the ray origins: dark blue
#p.add_points(pv.PolyData(ray_origins), color='#004887')

# plotting
p.show()


Widget(value='<iframe src="http://localhost:54220/index.html?ui=P_0x220efc27510_0&reconnect=auto" class="pyvis…

## 02. Create Various Performance matrices (Lattices in topogenesis terminology)
1. Quiteness : I have added a few points in Rhino as potential noise sources and the argmin distances from these points are considered as the level of quiteness for that voxel. Note quiteness is used and not dB levels since we are not calculating any sound insulations etc.
2. Distance from Facades : Distance from facades N,S,E,W. If the users have a priority to configure zones close to a specific facade 
3. Daylight : Simplified daylight analysis values for the voxels 
4. Distance from Lobby : It is the distance of the voxels from the Lobby entrances 
5. Distance from an Core: It is the distance of the voxels from the Core entrances 

In [6]:
class PerformanceLattice:
    def __init__(self, base_lattice):
        self.base_lattice = base_lattice
        self.north_distance = np.max(base_lattice.centroids.T[1])
        self.south_distance = np.min(base_lattice.centroids.T[1])
        self.east_distance = np.max(base_lattice.centroids.T[0])
        self.west_distance = np.min(base_lattice.centroids.T[0])

    def create_distance_lattice(self, point_lattice:np.array) -> None:
        """
        Create a distance based lattice  on the point sources and initial lattice.
        The resulting distance is arg min values from the closest point
        """
        # Create availability lattice
        init_lattice = self.base_lattice
        availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
        voxel_coordinates = availability_lattice_voxels.centroids

        # Flatten the initial lattice
        flattened_lattice = self.base_lattice.flatten()

        # Compute Euclidean distances
        eucledian_distance = sc.spatial.distance.cdist(point_lattice, voxel_coordinates)
        distance_from_each_source = eucledian_distance.T

        # Compute the average quiteness values
        average_distance_indexing = np.argmin(distance_from_each_source, axis=1)
        average_distance_values = [branch[index] for branch, index in zip(distance_from_each_source, average_distance_indexing)]

        # Compute the quiteness lattice
        distance_lattice_padded = np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(average_distance_values))])
        distance_lattice_np = distance_lattice_padded.reshape(self.base_lattice.shape)

        # Convert to lattice format
        distance_lattice = tg.to_lattice(distance_lattice_np, distance_lattice_np.shape)

        return distance_lattice
    
    def create_facade_lattice(self, direction:str) -> None:
        if direction == 'north':
            evaluation_value = self.north_distance
            distance_lattice = evaluation_value- self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape)
        elif direction == 'south':
            evaluation_value = self.south_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape))
        elif direction == 'east':
            evaluation_value = self.east_distance
            distance_lattice = evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape)
        elif direction == 'west':
            evaluation_value = self.west_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape))
        return distance_lattice
    
    def distance_from_outermostvoxel (self):
        return self.base_lattice

In [7]:
def csv_nparray(file_name:str) -> np.array:
    """
    Convert a csv file to numpy array
    """
    Complete_file = pd.read_excel(file_name, sheet_name=0,engine='openpyxl',header = None )
    array_excel = Complete_file.to_numpy()
    point_array = array_excel.T
    return point_array
# Numpy array of points indicating source of noise
Noise_sources = csv_nparray('Noise_sources.xlsx')
Core_centers = csv_nparray('Core_Centers.xlsx')

# Create Performance Lattice creator class
Performance_Creator = PerformanceLattice(lattice+1)
Quiteness_lattice = Performance_Creator.create_distance_lattice(Noise_sources)
Coredis_lattice = Performance_Creator.create_distance_lattice(Core_centers)
East_facade_lattice = Performance_Creator.create_facade_lattice('east')
West_facade_lattice = Performance_Creator.create_facade_lattice('west')
North_facade_lattice = Performance_Creator.create_facade_lattice('north')
South_facade_lattice = Performance_Creator.create_facade_lattice('south')
#Closeness to Facade can be equated to daylight availability
Daylight_simplified = Performance_Creator.distance_from_outermostvoxel()


In [8]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data Select whichever analysis you want to display
grid.cell_data["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
# Add your points to plot 
p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)

p.show()

Widget(value='<iframe src="http://localhost:54220/index.html?ui=P_0x220efb0e090_1&reconnect=auto" class="pyvis…

## 03. Create Desirability lattices ( Perform MCDA on the  based on the various performance matrices developed in the previous step ) ##

In [9]:
# Collect all the Performance matrice
array_all_performance_matrices = [Quiteness_lattice.flatten(),Coredis_lattice.flatten(), East_facade_lattice.flatten(), West_facade_lattice.flatten(), North_facade_lattice.flatten(), South_facade_lattice.flatten()]
performance_matrix = np.array(array_all_performance_matrices).T

In [10]:
class DesirabilityLattice:
    def __init__(self, performance_matrix:np.array):
        self.performance_matrix = performance_matrix
    
    def topsis (self, objectives_array, weights_array, criteria_array):
        """
        TOPSIS method to calculate the desirability of each voxel
        """
        decision_matrix = sk.mkdm(
        self.performance_matrix,
        objectives = objectives_array,
        weights = weights_array,
        criteria = criteria_array)

        pipe = mkpipe(
        invert_objectives.NegateMinimize(),
        scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix
        scalers.SumScaler(target="weights"),  # and this transform the weights
        similarity.TOPSIS(),
        )
        rank = pipe.evaluate(decision_matrix)
        mcdm_result= rank.e_.similarity.reshape(lattice.shape)
        
        return mcdm_result


In [18]:
criteria_array = ["quietness", "coredis", "eastfacade", "westfacade", "northfacade", "southfacade"]
objectives_array = [max,min,max,max,max,max]
decisionmaker = DesirabilityLattice(performance_matrix)

#Zone1 parameters
Z1_weights_array = [0.5, 0.5, 0, 0, 0.6,0]
Z1_Desirability_lattice = decisionmaker.topsis(objectives_array, Z1_weights_array, criteria_array)

In [ ]:
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data Select whichever analysis you want to display
grid.cell_data["Zone 1 Desirability Lattice"] = Z1_Desirability_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes
# Matplot lib colormaps https://matplotlib.org/stable/users/explain/colors/colormaps.html

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="BrBG", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
# Add your points to plot 
p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)

p.show()

Widget(value='<iframe src="http://localhost:54220/index.html?ui=P_0x2208f051d50_9&reconnect=auto" class="pyvis…

## 04. Initiate Agent classes ##

## 05. Run Agent Based Simulation ##